In [1]:
print("""
@File         : CH07.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-09-02 09:25:14
@Email        : cuixuanstephen@gmail.com
@Description  : 使用 pandas 操作 Excel 文件
""")


@File         : CH07.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-09-02 09:25:14
@Email        : cuixuanstephen@gmail.com
@Description  : 使用 pandas 操作 Excel 文件



In [2]:
%cd ../

d:\Data-Analysis-and-Science\PY4XL


In [3]:
import pandas as pd

In [4]:
df = pd.read_excel('sales_data/new/January.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9493 entries, 0 to 9492
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    9493 non-null   object        
 1   store             9493 non-null   object        
 2   status            9493 non-null   object        
 3   transaction_date  9493 non-null   datetime64[ns]
 4   plan              9493 non-null   object        
 5   contract_type     9493 non-null   object        
 6   amount            9493 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 519.3+ KB


In [5]:
df = pd.read_excel('xl/stores.xlsx', sheet_name='2019', skiprows=1, usecols='B:F')
df

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING
2,Chicago,4,Katelin,2020-01-31,NaN
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Store      6 non-null      object        
 1   Employees  6 non-null      int64         
 2   Manager    6 non-null      object        
 3   Since      5 non-null      datetime64[ns]
 4   Flagship   5 non-null      object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 368.0+ bytes


除了 `Flagship` 这一列，其他列看起来都还不错。`Flagship` 的数据类型应该是 `bool` 而不是 `object`。要修正这一问题，需要提供一个转换函数来处理某列中发生冲突的单元格。（除了编写 `fix_missing` 函数，也可以提供一个 `lambda` 表达式。）

In [8]:
def fix_missing(x):
    return False if x in ["", "MISSING"] else x


df = pd.read_excel('xl/stores.xlsx', sheet_name='2019', skiprows=1, usecols='B:F',
                   converters={'Flagship': fix_missing})

df

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,False
2,Chicago,4,Katelin,2020-01-31,False
3,Boston,5,Georgiana,2017-04-01,True
4,Washington DC,3,Evan,NaT,False
5,Las Vegas,11,Paul,2020-01-06,False


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Store      6 non-null      object        
 1   Employees  6 non-null      int64         
 2   Manager    6 non-null      object        
 3   Since      5 non-null      datetime64[ns]
 4   Flagship   6 non-null      bool          
dtypes: bool(1), datetime64[ns](1), int64(1), object(2)
memory usage: 326.0+ bytes


`read_excel` 函数也可以接受一个工作表名称列表。在这种情况下，它返回的是以 DataFrame 为值、工作表名称为键的一个字典。要读入所有的工作表，你需要传递参数 `sheet_name=None`。另外，注意这里使用了 `usecols` 的一种变体，传递了表的列名作为参数：

In [10]:
sheets = pd.read_excel('xl/stores.xlsx', sheet_name=['2019', '2020'],
                       skiprows=1, usecols=['Store', 'Employees'])

In [13]:
sheets['2019'].head(2)

,Store,Employees
0,New York,10
1,San Francisco,12


为了处理 `NaN`，可以把 `na_values` 和 `keep_default_na` 结合起来。

In [15]:
df = pd.read_excel('xl/stores.xlsx', sheet_name='2019',
                   skiprows=1, usecols='B,C,F', skipfooter=2, 
                   na_values='MISSING', 
                   keep_default_na=False)

df

,Store,Employees,Flagship
0,New York,10,False
1,San Francisco,12,NaN
2,Chicago,4,
3,Boston,5,True


pandas 还提供了一种读取 Excel 文件的方法，那就是使用 `ExcelFile` 类。如果你想从旧式的 xls 文件中读取多张工作表，`ExcelFile` 就会发挥作用。在这种情况下，`ExcelFile` 可以防止 pandas 多次读取整个文件，从而获得较快的速度。由于 `ExcelFile` 可以被用作上下文管理器（参见“[上下文管理器和 `with` 语句](https://github.com/JPL-JUNO/Data-Analysis-and-Science/blob/main/PY4XL/others/with.md)”），因而文件可以被正确关闭。

先来实际了解一下 `ExcelFile` 类：

In [16]:
with pd.ExcelFile('xl/stores.xls') as f:
    df1 = pd.read_excel(f, '2019', skiprows=1, usecols='B:F', nrows=2)
    df2 = pd.read_excel(f, '2020', skiprows=1, usecols='B:F', nrows=2)
    
df1

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING


In [17]:
df2

,Store,Employees,Manager,Since,Flagship
0,New York,11,Sarah,2018-07-20,False
1,San Francisco,10,Neriah,2019-11-02,True


也可以通过 `ExcelFile` 访问所有工作表的名称：

In [19]:
stores = pd.ExcelFile('xl/stores.xlsx')
stores.sheet_names

['2019', '2020', '2019-2020']

In [20]:
url = ("https://raw.githubusercontent.com/fzumstein/python-for-excel/1st-edition/xl/stores.xlsx")
pd.read_excel(url, skiprows=1, usecols='B:F', nrows=2)

,Store,Employees,Manager,Since,Flagship
0,New York,10,Sarah,2018-07-20,False
1,San Francisco,12,Neriah,2019-11-02,MISSING


>
> 如果你使用的是低于 1.3 版本的 pandas，那么在使用 `read_excel` 函数或 `ExcelFile` 类读取 `xlsb` 文件时就需要显式地指定引擎：

In [22]:
pd.read_excel('xl/stores.xlsb', engine='pyxlsb')

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,Store,Employees,Manager,Since,Flagship
1,NaN,New York,10,Sarah,43301,False
2,NaN,San Francisco,12,Neriah,43771,MISSING
3,NaN,Chicago,4,Katelin,43861,NaN
4,NaN,Boston,5,Georgiana,42826,True
5,NaN,Washington DC,3,Evan,NaN,False
6,NaN,Las Vegas,11,Paul,43836,False


|参数|描述|
|---|---|
|`sheet_name`|除了提供工作表名称，你也可以提供工作表的索引（从 0 开始），比如 `sheet_name=0`。如果将参数设置为 `sheet_name=None`，则 pandas 会读取整个工作簿并以 `{"sheetname": df}` 的形式返回一个字典。要读取指定的多张工作表，可以传递一个工作表名称或索引的列表|
|`skiprows`|你可以借此跳过指定数量的行|
|`usecols`|如果 Excel 文件包含列标题，就通过传递列标题列表选择指定列，比如 `["Store", "Employees"]`。另外，也可以传递列的索引列表，比如 `[1, 2]`。或者 Excel 列名的字符串（不是列表）也可以包含列区域，比如 `"B:D, G"`。你还可以传递一个函数：如果想只包含以 `Manager` 开始的列，则可以将参数设置为 `usecols=lambda x: x.startwith("Manager")`|
|`nrows`|想要读取的行数|
|`index_col`|指定将作为索引的列接受列名或列索引，比如 `index_col=0`。如果提供了包含多列的列表，则会创建层次索引|
|`header`|如果设置为 `header=None`，而未通过 `names` 参数提供列名，则会使用默认的整数列标题。如果提供的是索引的列表，则会创建层次列标题|
|`names`|提供列名称列表|
|`na_values`|在默认情况下，pandas 会将这些值解释为 `NaN`：`空单元格、#NA、NA、null、#N/A、N/A、NaN、n/a、-NaN、1.#IND、nan、#N/A N/A、-1.#QNAN、-nan、NULL、-1.#IND、<NA> 和 1.#QNAN`。如果需要往这些值中添加一个或多个值，则可以通过 `na_values` 来提供|
|`keep_default_na`|如果希望忽略 pandas 默认解释为 `NaN` 的值，则可以将参数设置为 `keep_default_na=False`|
|`convert_float`|在默认情况下，Excel 会在内部将所有数字都以浮点型保存，pandas 会将带有无意义的小数点的数字转换为整数。如果想改变这种行为，则可以将参数设置为 `convert_float=False`（可能会获得少许性能提升）|
|`converters`|可以为各列提供一个函数来转换其中的值。如果要将某一列中的文本转换为大写，则可以将参数设置为：`converters={"column_name": lambda x: x.upper()}`|

## `to_excel` 方法和 `ExcelWriter` 类
用 pandas 写入 Excel 文件的最简单的方法是使用 DataFrame 的 `to_excel` 方法。你可以用它来指定要将 DataFrame 写入哪些工作表的哪些单元格，以及是否需要包含列标题和 DataFrame 索引。对于 `np.nan` 和 `np.inf` 这类在 Excel 中没有等价表达方式的值，你也可以用这个方法来告诉 pandas 如何处理。

In [25]:
import numpy as np
import datetime as dt

data = [[dt.datetime(2020, 1, 2, 10, 13), 2.222, 1, True],
        [dt.datetime(2020, 1, 2), np.nan, 2, False],
        [dt.datetime(2020, 1, 2), np.inf, 3, True],]

df = pd.DataFrame(data=data, columns=['Dates', 'Floats', 'Integers', 'Booleans'])
df.index.name = 'index'
df

,Dates,Floats,Integers,Booleans
index,,,,
0,2020-01-02 10:13:00,2.222,1,True
1,2020-01-02 00:00:00,NaN,2,False
2,2020-01-02 00:00:00,inf,3,True


In [26]:
df.to_excel('res/written_with_pandas.xlsx', sheet_name='Output', startrow=1, startcol=1, 
            index=True, header=True, na_rep='<NA>', inf_rep='<INF>')

如果想将多个 DataFrame 写入同一张或多张工作表，则需要使用 `ExcelClass` 类。下面的例子分 3 次将同一个 DataFrame 写入工作表，前两次写入了工作表 1 的两个位置，第三次写入了工作表 2:

In [27]:
with pd.ExcelWriter('res/written_with_pandas2.xlsx') as writer:
    df.to_excel(writer, sheet_name='Sheet1', startrow=1, startcol=1)
    df.to_excel(writer, sheet_name='Sheet1', startrow=10, startcol=1)
    df.to_excel(writer, sheet_name='Sheet2')

由于将 `ExcelClass` 用作了上下文管理器，因此当文件离开上下文管理器时（也就是离开由缩进定义的代码块时）会被自动写入磁盘。如果不像这样写的话，则必须显式地调用 `writer.save()`。

|参数|描述|
|---|---|
|`sheet_name`|要写入的工作表名称|
|`startrow` 和 `startcol`|`startrow` 是 DataFrame 中会被写入的第一行，而 `startcol` 是第一列。这里的索引是从 0 开始的，如果想要将 DataFrame 写入单元格 B3，则需要将参数设置为 `startrow=2` 和 `startcol=1`|
|`index` 和 `header`|如果想隐藏索引和 / 或标题，则需要将参数设置为 `index=False` 和 `header=False`|
|`na_rep` 和 `inf_rep`|在默认情况下，np.nan 会被转换为空单元格，而 NumPy 用来表示无穷的 `np.inf` 会被转换为字符串 `inf`。利用这两个参数可以修改默认行为|
|`freeze_panes`|通过提供一个元组来冻结前几行和前几列，比如 (2, 1) 会冻结前两行和第一列|

## 使用 pandas 处理 Excel 文件的局限性

使用 pandas 接口读写简单的 Excel 文件非常好用，但是也有一些局限性。

- 将 DataFrame 写入文件时，无法将标题或图表也写入文件。
- 无法修改 Excel 中标题和索引的默认格式。
- 在读取文件时，pandas 会自动转换错误单元格（比如将 `#REF!` 或 `#NUM!` 转换为 `NaN`），从而使你无法在工作表中查找特定的错误。
- 处理大型 Excel 文件时可能需要额外的设置，这种情况下直接使用读写包更容易操作。

## 小结

pandas 的好处在于它为处理所有受支持的 Excel 文件格式提供了统一的接口，无论是 xls、xlsx、xlsm 和 xlsb 之中的哪一种格式。这让我们在读取包含 Excel 文件的目录、聚合数据
和总结数据生成 Excel 报表都更加容易。

不过 pandas 并非独自承受其重：在底层，pandas 会选择某个读写包来完成工作。